In [5]:
import cv2
import os
from tkinter import Tk, Button, Label, OptionMenu, StringVar
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
import hashlib
from random import randint

# Create necessary directories
if not os.path.exists('dataset'):
    os.makedirs('dataset')
    print("Created directory: dataset")

# Initialize camera variables
cap = None
camera_index = 0  
image_id = 1

def get_camera_indexes():
    indexes = []
    index = 0
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        indexes.append(index)
        cap.release()
        index += 1
    return indexes

def apply_augmentations(frame):
    augmented_images = []
    # Convert to RGB format for PIL
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Original image
    augmented_images.append(img)
    # Rotate
    for angle in [90, 180, 270]:
        rotated = img.rotate(angle)
        augmented_images.append(rotated)
    # Zoom in
    zoomed_in = img.resize((int(img.width * 1.5), int(img.height * 1.5)))
    augmented_images.append(zoomed_in)
    # Zoom out
    zoomed_out = img.resize((int(img.width * 0.5), int(img.height * 0.5)))
    augmented_images.append(zoomed_out)
    # Blur
    blurred = img.filter(ImageFilter.GaussianBlur(radius=5))
    augmented_images.append(blurred)
    # Crop
    left = randint(0, 20)
    upper = randint(0, 20)
    right = img.width - randint(0, 20)
    lower = img.height - randint(0, 20)
    cropped = img.crop((left, upper, right, lower))
    augmented_images.append(cropped)
    # Flip
    flipped_horizontal = img.transpose(Image.FLIP_LEFT_RIGHT)
    augmented_images.append(flipped_horizontal)
    
    flipped_vertical = img.transpose(Image.FLIP_TOP_BOTTOM)
    augmented_images.append(flipped_vertical)
    for factor in [0.5, 1.5]:  
        brightness_enhancer = ImageEnhance.Brightness(img)
        enhanced = brightness_enhancer.enhance(factor)
        augmented_images.append(enhanced)

    return augmented_images

def capture_image():
    global image_id
    print("Capture image function called.")

    ret, frame = cap.read()

    if ret:
        hash_object = hashlib.md5(frame.tobytes())
        hash_str = hash_object.hexdigest()
        
        augmented_images = apply_augmentations(frame)

        for i, img in enumerate(augmented_images):
            image_name = f"IMG_{image_id:04d}_JPG.rf.{hash_str}_{i}.jpg"
            image_path = f'dataset/2/{image_name}'

            img.save(image_path)
            print(f'Saved image to {image_path}')

        print(f'Image set for ID {image_id} captured and saved.')

        image_id += 1
    else:
        print("Failed to capture frame from camera.")

def select_camera():
    global cap, camera_index
    camera_index = int(camera_var.get())
    
    if cap is not None:
        cap.release()  

    update_camera_label()

def update_camera_label():
    camera_name = f'Camera ID: {camera_index}'
    camera_label.config(text=camera_name)

def start_camera():
    global cap
    cap = cv2.VideoCapture(camera_index)  

    if not cap.isOpened():
        print(f"Error: Unable to access camera {camera_index}.")
        return

    def update_frame():
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            camera_display.imgtk = imgtk  
            camera_display.config(image=imgtk)
        else:
            print("Error: Unable to read frame from camera.")

        camera_display.after(10, update_frame)  

    update_frame()  

root = Tk()
root.title("Camera Capture Tool")

camera_indexes = get_camera_indexes()

if camera_indexes:
    camera_var = StringVar(value=camera_indexes[0])
    camera_dropdown = OptionMenu(root, camera_var, *camera_indexes, command=lambda _: select_camera())
    camera_dropdown.pack()
    camera_label = Label(root, text=f'Camera ID: {camera_indexes[0]}')
    camera_label.pack()
    start_camera_button = Button(root, text="Start Camera", command=start_camera)
    start_camera_button.pack()
    camera_display = Label(root)
    camera_display.pack()
    capture_button = Button(root, text="Capture Image", command=capture_image)
    capture_button.pack()
else:
    Label(root, text="No cameras found.").pack()

root.mainloop()


Capture image function called.
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_0.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_1.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_2.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_3.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_4.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_5.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_6.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_7.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_8.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_9.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_10.jpg
Saved image to dataset/2/IMG_0001_JPG.rf.b554b4c7af02334e8365b594f89bdfd1_11.jpg
Image s